<a href="https://colab.research.google.com/github/len-rtz/2024-10-15-Master_Digital_Sciences-Linked_Open_Data_and_Knowledge_Graphs_WiSe_2024_20245/blob/main/encoder/Finetuning_PEFT_encoder-modernBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# change direcotry
import sys
sys.path.append('/Users/lenap/Documents/Git/ArgumentMining')
print(sys.path)

['/Users/lenap/Documents/Git', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa', '/opt/anaconda3/lib/python3.12/site-packages/setuptools/_vendor', '/Users/lenap/Documents/Git/Argument-Mining']


In [ ]:
from db.queries import get_training_data, get_test_data

# Load training data
claims_train, premises_train, relationships_train = get_training_data()

# Load test data
claims_test, premises_test, relationships_test = get_test_data()

In [ ]:
import pandas as pd

# Create pairs of claims and premises
debate_pairs = []

for i in range(len(claims_train)):
    debate_pairs.append({
        "claim": claims_train[i].text,
        "premise": premises_train[i].text,
        "stance": relationships_train[i],
    })

# Create final DataFrame
train_data = pd.DataFrame(debate_pairs)

print(train_data.head())

                                               claim  \
0  This house believes that the sale of violent v...   
1  This house supports the one-child policy of th...   
2  This house would permit the use of performance...   
3  This house would make physical education compu...   
4  This house believes in the use of affirmative ...   

                                             premise      stance  
0  video game violence is not related to serious ...  stance_con  
1         The policy had proved remarkably effective  stance_pro  
2  The use of drugs to enhance performance is con...  stance_con  
3  Frequent and regular physical exercise boosts ...  stance_pro  
4  In some countries which have laws on racial eq...  stance_con  


In [ ]:
# Remove 'stance_' prefix for simplicity
train_data['stance'] = train_data['stance'].str.replace('stance_', '')

print(train_data.head())

                                               claim  \
0  This house believes that the sale of violent v...   
1  This house supports the one-child policy of th...   
2  This house would permit the use of performance...   
3  This house would make physical education compu...   
4  This house believes in the use of affirmative ...   

                                             premise stance  
0  video game violence is not related to serious ...    con  
1         The policy had proved remarkably effective    pro  
2  The use of drugs to enhance performance is con...    con  
3  Frequent and regular physical exercise boosts ...    pro  
4  In some countries which have laws on racial eq...    con  


# Finetune ModernBERT

In [ ]:
import torch
import pandas as pd
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    PeftModel,
    PeftConfig
)

In [ ]:
model_name = "answerdotai/ModernBERT-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Iterate through the model's named modules and print the names of linear layers
print("Available linear layers in the ModernBERT model:")
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(name)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Available linear layers in the ModernBERT model:
model.layers.0.attn.Wqkv
model.layers.0.attn.Wo
model.layers.0.mlp.Wi
model.layers.0.mlp.Wo
model.layers.1.attn.Wqkv
model.layers.1.attn.Wo
model.layers.1.mlp.Wi
model.layers.1.mlp.Wo
model.layers.2.attn.Wqkv
model.layers.2.attn.Wo
model.layers.2.mlp.Wi
model.layers.2.mlp.Wo
model.layers.3.attn.Wqkv
model.layers.3.attn.Wo
model.layers.3.mlp.Wi
model.layers.3.mlp.Wo
model.layers.4.attn.Wqkv
model.layers.4.attn.Wo
model.layers.4.mlp.Wi
model.layers.4.mlp.Wo
model.layers.5.attn.Wqkv
model.layers.5.attn.Wo
model.layers.5.mlp.Wi
model.layers.5.mlp.Wo
model.layers.6.attn.Wqkv
model.layers.6.attn.Wo
model.layers.6.mlp.Wi
model.layers.6.mlp.Wo
model.layers.7.attn.Wqkv
model.layers.7.attn.Wo
model.layers.7.mlp.Wi
model.layers.7.mlp.Wo
model.layers.8.attn.Wqkv
model.layers.8.attn.Wo
model.layers.8.mlp.Wi
model.layers.8.mlp.Wo
model.layers.9.attn.Wqkv
model.layers.9.attn.Wo
model.layers.9.mlp.Wi
model.layers.9.mlp.Wo
model.layers.10.attn.Wqkv
model

In [ ]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [ ]:
# Task definitions
TASKS = {
    "adu_identification": {
        "num_labels": 2,  # Yes/No - contains ADU
        "id2label": {0: "No", 1: "Yes"},
        "label2id": {"No": 0, "Yes": 1}
    },
    "adu_classification": {
        "num_labels": 2,  # claim or premise
        "id2label": {0: "claim", 1: "premise"},
        "label2id": {"claim": 0, "premise": 1}
    },
    "stance_classification": {
        "num_labels": 2,  # pro or con
        "id2label": {0: "con", 1: "pro"},
        "label2id": {"con": 0, "pro": 1}
    },
    "relationship_identification": {
        "num_labels": 2,  # supportive or contradictory
        "id2label": {0: "contradictory", 1: "supportive"},
        "label2id": {"contradictory": 0, "supportive": 1}
    }
}

# Data Formatting

In [ ]:
def format_for_argument_mining_modernbert(df):
    formatted_data = []
    for _, row in df.iterrows():
        # Task 1: ADU Identification (claim)
        claim_adu_sample = {
            "task": "adu_identification",
            "text": row['claim'],
            "label": 1  # 1 for contains ADU
        }
        # Task 1: ADU Identification (premise)
        premise_adu_sample = {
            "task": "adu_identification",
            "text": row['premise'],
            "label": 1  # 1 for contains ADU
        }
        # Task 2: ADU Classification (claim)
        adu_class_sample_claim = {
            "task": "adu_classification",
            "text": row['claim'],
            "label": 0  # 0 for claim
        }
        # Task 2: ADU Classification (premise)
        adu_class_sample_premise = {
            "task": "adu_classification",
            "text": row['premise'],
            "label": 1  # 1 for premise
        }
        # Task 3: Stance Classification
        stance_sample = {
            "task": "stance_classification",
            "text": f"{row['claim']} {row['premise']}",  # Using space as separator for ModernBERT
            "label": 1 if row['stance'] == 'pro' else 0  # 1 for 'pro', 0 for 'con'
        }
        # Task 4: Relationship identification
        relationship_sample = {
            "task": "relationship_identification",
            "text": f"{row['claim']} {row['premise']}",  # Using space as separator for ModernBERT
            "label": 1 if row['stance'] == 'pro' else 0  # 1 for supportive, 0 for contradictory
        }
        # Add all tasks to our dataset
        formatted_data.extend([
            claim_adu_sample,
            premise_adu_sample,
            adu_class_sample_claim,
            adu_class_sample_premise,
            stance_sample,
            relationship_sample
        ])
    return formatted_data

In [ ]:
def tokenize_function_modernbert(examples, tokenizer, max_length=512):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )

# Language Model configuration

In [ ]:
def setup_modernbert_model(model_name="answerdotai/ModernBERT-base", num_labels=2):
    """
    Loads a pre-trained ModernBERT model and tokenizer for sequence classification.
    """
    try:
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            problem_type="single_label_classification"
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        raise

# PEFT LoRa configuration

In [ ]:
def configure_peft_modernbert(model):
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        # UPDATED target_modules based ModernBERT architecture
        target_modules=["Wqkv", "Wo", "Wi"], # Targeting attention and MLP inner layers
        bias="none",
    )
    try:
        model = get_peft_model(model, peft_config)
    except ValueError as e:
        print(f"Error configuring PEFT: {e}")
        print("\nEnsure the target_modules in LoraConfig exactly match the names of the linear layers within your model's architecture.")
        print("You can inspect model.named_modules() to confirm.")
        raise # Re-raise the exception after providing more context

    print_trainable_parameters(model)
    return model

# print_trainable_parameters (unchanged)
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )


In [ ]:
def setup_training_modernbert(model, train_dataset, eval_dataset, output_dir="./argument-mining-modernbert"):
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=16,
        learning_rate=2e-5,
        weight_decay=0.01,
        logging_steps=20,
        save_strategy="epoch",
        warmup_ratio=0.1,
        eval_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy"
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = torch.argmax(torch.tensor(logits), dim=-1)
        accuracy = (predictions == torch.tensor(labels)).float().mean().item()
        return {"accuracy": accuracy}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    return trainer

In [ ]:
def train_argument_mining_modernbert(train_data, model_name="answerdotai/ModernBERT-base"):
    # Split train data for cross-validation
    train_df, eval_df = train_test_split(train_data, test_size=0.2, random_state=42)

    # Format data for ModernBERT training
    formatted_train_data = format_for_argument_mining_modernbert(train_df)
    formatted_eval_data = format_for_argument_mining_modernbert(eval_df)

    task_models = {}
    for task in ["adu_identification", "adu_classification", "stance_classification", "relationship_identification"]:
        # Filter data for specific task
        task_data = [item for item in formatted_train_data if item["task"] == task]
        task_eval_data = [item for item in formatted_eval_data if item["task"] == task]

        # Skip if no data for task
        if not task_data or not task_eval_data:
            print(f"Skipping task {task} due to insufficient data.")
            continue

        # Determine num_labels for the current task
        # Ensure num_labels matches the TASKS dictionary definition for each task
        num_labels = TASKS[task]["num_labels"]

        # Setup model and tokenizer for this task
        model, tokenizer = setup_modernbert_model(model_name, num_labels=num_labels)

        # Create HF Dataset
        hf_train_dataset = Dataset.from_list(task_data)
        hf_eval_dataset = Dataset.from_list(task_eval_data)

        # Apply tokenizer
        train_dataset = hf_train_dataset.map(
            lambda x: tokenize_function_modernbert(x, tokenizer),
            batched=True,
            remove_columns=["text", "task"]
        )
        eval_dataset = hf_eval_dataset.map(
            lambda x: tokenize_function_modernbert(x, tokenizer),
            batched=True,
            remove_columns=["text", "task"]
        )

        # Configure PEFT/LoRA
        model = configure_peft_modernbert(model)

        # Setup training
        trainer = setup_training_modernbert(
            model, train_dataset, eval_dataset, output_dir=f"./argument-mining-modernbert-{task}"
        )

        # Train the model
        trainer.train()

        # Save the model and tokenizer
        peft_model_id = f"argument-mining-modernbert-{task}"
        trainer.model.save_pretrained(peft_model_id)
        tokenizer.save_pretrained(peft_model_id)
        print(f"Model for task {task} saved to {peft_model_id}")

        # Store model for inference
        task_models[task] = {"model": model, "tokenizer": tokenizer, "model_id": peft_model_id}

    return task_models

# Training

In [ ]:
# Train all task models
task_models = train_argument_mining_modernbert(train_data)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/29264 [00:00<?, ? examples/s]

Map:   0%|          | 0/7318 [00:00<?, ? examples/s]

trainable params: 1691138 || all params: 151297540 || trainable%: 1.12


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
